#Lab: process data files with Spark

In this exercise you will parse a set of activation records in JSON format to extract the account numbers and model names.
Spark is commonly used for ETL (Extract/Transform/Load) operations. Sometimes data is stored in line-oriented records,
like the web logs in the previous exercise, but sometimes the data is in a multi-line format that must be processed as a whole
le. In this exercise you will practice working with le-based instead of line-based formats.

## Put your data on Databricks

Download the activations.tgz le from Blackboard and unpack with tar xvzf activations.tgz. Review the data. Each
JSON le contains data for all the devices activated by customers during a specic month.

--> 1. Download the json activations.zip le from Blackboard. This will download it to the machine you're using. To get
the zip le onto Databricks, we'll use the Databricks user interface (UI).

--> 2. Log into Databricks so you can see the \Welcome to databricks" screen. (If you're already logged in, you can get back
to this screen by clicking the topmost icon on the left.)

--> 3. There should be three columns, the middle of which is titled \Import & Explore Data". Click on the \click to browse"
link above this and select the json activations.zip le you downloaded to your system. Wait for the le to upload,
but do not click on anything else on that page after the upload nishes.

--> 4. By default, the UI uploads to /FileStore/tables/. Back in a notebook, use the dbutils.fs.ls command to check
that the le has been uploaded.

In [0]:
dbutils.fs.ls("dbfs:/FileStore/tables/")

Out[1]: [FileInfo(path='dbfs:/FileStore/tables/Dummy.txt', name='Dummy.txt', size=108),
 FileInfo(path='dbfs:/FileStore/tables/Iris_data/', name='Iris_data/', size=0),
 FileInfo(path='dbfs:/FileStore/tables/json_activations.zip', name='json_activations.zip', size=8411369)]

--> 5. We need to extract this zip archive. Since the dbutils toolkit doesn't provide an unzip command, you need to copy the
le to the driver node, extract there using a shell command, and put the extracted contents back into DBFS. So, rst
copy the json activations.zip from DBFS to the /tmp directory on your driver node using dbutils and verify the
le has been copied.

In [0]:
dbutils.fs.cp("dbfs:/FileStore/tables/json_activations.zip", "file:/tmp")

Out[56]: True

In [0]:
dbutils.fs.ls("file:/tmp")

Out[57]: [FileInfo(path='file:/tmp/driver-daemon.pid', name='driver-daemon.pid', size=4),
 FileInfo(path='file:/tmp/systemd-private-6c16a50f3557429f9ac5d62c133a471a-apache2.service-KNfXjO/', name='systemd-private-6c16a50f3557429f9ac5d62c133a471a-apache2.service-KNfXjO/', size=4096),
 FileInfo(path='file:/tmp/chauffeur-daemon-params', name='chauffeur-daemon-params', size=22),
 FileInfo(path='file:/tmp/driver-env.sh', name='driver-env.sh', size=3029),
 FileInfo(path='file:/tmp/.Test-unix/', name='.Test-unix/', size=4096),
 FileInfo(path='file:/tmp/.ICE-unix/', name='.ICE-unix/', size=4096),
 FileInfo(path='file:/tmp/RtmpuQWsTm/', name='RtmpuQWsTm/', size=4096),
 FileInfo(path='file:/tmp/.font-unix/', name='.font-unix/', size=4096),
 FileInfo(path='file:/tmp/.XIM-unix/', name='.XIM-unix/', size=4096),
 FileInfo(path='file:/tmp/chauffeur-env.sh', name='chauffeur-env.sh', size=156),
 FileInfo(path='file:/tmp/Rserv/', name='Rserv/', size=4096),
 FileInfo(path='file:/tmp/tmp.KWGvMO9xaY', name='tmp.KWGvMO9xaY', size=0),
 FileInfo(path='file:/tmp/custom-spark.conf', name='custom-spark.conf', size=166),
 FileInfo(path='file:/tmp/systemd-private-6c16a50f3557429f9ac5d62c133a471a-ntp.service-oFk8KO/', name='systemd-private-6c16a50f3557429f9ac5d62c133a471a-ntp.service-oFk8KO/', size=4096),
 FileInfo(path='file:/tmp/driver-daemon-params', name='driver-daemon-params', size=19),
 FileInfo(path='file:/tmp/systemd-private-6c16a50f3557429f9ac5d62c133a471a-systemd-resolved.service-PGrkwt/', name='systemd-private-6c16a50f3557429f9ac5d62c133a471a-systemd-resolved.service-PGrkwt/', size=4096),
 FileInfo(path='file:/tmp/hsperfdata_root/', name='hsperfdata_root/', size=4096),
 FileInfo(path='file:/tmp/chauffeur-daemon.pid', name='chauffeur-daemon.pid', size=4),
 FileInfo(path='file:/tmp/.X11-unix/', name='.X11-unix/', size=4096),
 FileInfo(path='file:/tmp/json_activations.zip', name='json_activations.zip', size=8411369)]

--> 6. Use the UNIX command unzip to extract the contents. I.e. assuming success in Point 5, you should be able to run

which should return information about the creation of an activations directory and that various les have been
inflated into it. The -d option tells unzip to perform the extraction into tmp rather than your current working
directory. Conrm this has worked with an ls of the /tmp/ and /tmp/activations directories.

In [0]:
%sh

unzip -d /tmp/ /tmp/json_activations.zip

Archive: /tmp/json_activations.zip
 creating: /tmp/activations/
 inflating: /tmp/activations/2011-04.json 
 inflating: /tmp/activations/2013-10.json 
 inflating: /tmp/activations/2011-09.json 
 inflating: /tmp/activations/2011-02.json 
 inflating: /tmp/activations/2009-12.json 
 inflating: /tmp/activations/2010-10.json 
 inflating: /tmp/activations/2009-11.json 
 inflating: /tmp/activations/2014-03.json 
 inflating: /tmp/activations/2012-02.json 
 inflating: /tmp/activations/2013-07.json 
 inflating: /tmp/activations/2013-04.json 
 inflating: /tmp/activations/2010-11.json 
 inflating: /tmp/activations/2009-02.json 
 inflating: /tmp/activations/2012-05.json 
 inflating: /tmp/activations/2013-12.json 
 inflating: /tmp/activations/2009-07.json 
 inflating: /tmp/activations/2012-04.json 
 inflating: /tmp/activations/2009-03.json 
 inflating: /tmp/activations/2008-12.json 
 inflating: /tmp/activations/2011-05.json 
 inflating: /tmp/activations/2009-09.json 
 inflating: /tmp/activations/2012-03.json 
 inflating: /tmp/activations/2013-02.json 
 inflating: /tmp/activations/2011-06.json 
 inflating: /tmp/activations/2010-02.json 
 inflating: /tmp/activations/2008-11.json 
 inflating: /tmp/activations/2008-10.json 
 inflating: /tmp/activations/2013-09.json 
 inflating: /tmp/activations/2011-07.json 
 inflating: /tmp/activations/2013-06.json 
 inflating: /tmp/activations/2010-05.json 
 inflating: /tmp/activations/2014-01.json 
 inflating: /tmp/activations/2012-06.json 
 inflating: /tmp/activations/2012-12.json 
 inflating: /tmp/activations/2009-10.json 
 inflating: /tmp/activations/2010-01.json 
 inflating: /tmp/activations/2011-10.json 
 inflating: /tmp/activations/2012-07.json 
 inflating: /tmp/activations/2012-10.json 
 inflating: /tmp/activations/2012-11.json 
 inflating: /tmp/activations/2010-12.json 
 inflating: /tmp/activations/2011-11.json 
 inflating: /tmp/activations/2011-08.json 
 inflating: /tmp/activations/2013-11.json 
 inflating: /tmp/activations/2013-03.json 
 inflating: /tmp/activations/2009-04.json 
 inflating: /tmp/activations/2012-09.json 
 inflating: /tmp/activations/2010-03.json 
 inflating: /tmp/activations/2009-06.json 
 inflating: /tmp/activations/2014-02.json 
 inflating: /tmp/activations/2009-01.json 
 inflating: /tmp/activations/2011-01.json 
 inflating: /tmp/activations/2010-09.json 
 inflating: /tmp/activations/2010-04.json 
 inflating: /tmp/activations/2010-06.json 
 inflating: /tmp/activations/2013-01.json 
 inflating: /tmp/activations/2013-08.json 
 inflating: /tmp/activations/2011-12.json 
 inflating: /tmp/activations/2009-05.json 
 inflating: /tmp/activations/2013-05.json 
 inflating: /tmp/activations/2011-03.json 
 inflating: /tmp/activations/2010-07.json 
 inflating: /tmp/activations/2012-08.json 
 inflating: /tmp/activations/2012-01.json 
 inflating: /tmp/activations/2009-08.json 
 inflating: /tmp/activations/2010-08.json

In [0]:
%sh

ls /tmp/activations

2008-10.json
2008-11.json
2008-12.json
2009-01.json
2009-02.json
2009-03.json
2009-04.json
2009-05.json
2009-06.json
2009-07.json
2009-08.json
2009-09.json
2009-10.json
2009-11.json
2009-12.json
2010-01.json
2010-02.json
2010-03.json
2010-04.json
2010-05.json
2010-06.json
2010-07.json
2010-08.json
2010-09.json
2010-10.json
2010-11.json
2010-12.json
2011-01.json
2011-02.json
2011-03.json
2011-04.json
2011-05.json
2011-06.json
2011-07.json
2011-08.json
2011-09.json
2011-10.json
2011-11.json
2011-12.json
2012-01.json
2012-02.json
2012-03.json
2012-04.json
2012-05.json
2012-06.json
2012-07.json
2012-08.json
2012-09.json
2012-10.json
2012-11.json
2012-12.json
2013-01.json
2013-02.json
2013-03.json
2013-04.json
2013-05.json
2013-06.json
2013-07.json
2013-08.json
2013-09.json
2013-10.json
2013-11.json
2013-12.json
2014-01.json
2014-02.json
2014-03.json

--> 7. Create a DBFS directory activations in the /FileStore/ directory using the dbutils.fs.mkdirs command.

In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/activations/")
dbutils.fs.ls("dbfs:/FileStore/")

Out[67]: [FileInfo(path='dbfs:/FileStore/1gram-chi-sim-all-data', name='1gram-chi-sim-all-data', size=91613496),
 FileInfo(path='dbfs:/FileStore/activations/', name='activations/', size=0),
 FileInfo(path='dbfs:/FileStore/import-stage/', name='import-stage/', size=0),
 FileInfo(path='dbfs:/FileStore/iplist/', name='iplist/', size=0),
 FileInfo(path='dbfs:/FileStore/landsat_data.txt', name='landsat_data.txt', size=7785),
 FileInfo(path='dbfs:/FileStore/tables/', name='tables/', size=0)]

--> 8. Now you need to move (using the dbutils.fs.mv command) the contents of the local /tmp/activations directory
into your newly created DBFS /FileStore/activations directory so you can perform some Spark processing. If you
get an error when trying to use this command, you should look up its details so you can address the problem. Check
the contents of /FileStore/activations and conrm it contains the expected les.

In [0]:
dbutils.fs.mv("file:/tmp/activations/", "dbfs:/FileStore/activations/", True)

Out[69]: True

In [0]:
# checking the contents --> 

dbutils.fs.ls("dbfs:/FileStore/activations/")

Out[70]: [FileInfo(path='dbfs:/FileStore/activations/2008-10.json', name='2008-10.json', size=17958),
 FileInfo(path='dbfs:/FileStore/activations/2008-11.json', name='2008-11.json', size=69467),
 FileInfo(path='dbfs:/FileStore/activations/2008-12.json', name='2008-12.json', size=64783),
 FileInfo(path='dbfs:/FileStore/activations/2009-01.json', name='2009-01.json', size=85111),
 FileInfo(path='dbfs:/FileStore/activations/2009-02.json', name='2009-02.json', size=63749),
 FileInfo(path='dbfs:/FileStore/activations/2009-03.json', name='2009-03.json', size=81257),
 FileInfo(path='dbfs:/FileStore/activations/2009-04.json', name='2009-04.json', size=77942),
 FileInfo(path='dbfs:/FileStore/activations/2009-05.json', name='2009-05.json', size=84937),
 FileInfo(path='dbfs:/FileStore/activations/2009-06.json', name='2009-06.json', size=83592),
 FileInfo(path='dbfs:/FileStore/activations/2009-07.json', name='2009-07.json', size=81819),
 FileInfo(path='dbfs:/FileStore/activations/2009-08.json', name='2009-08.json', size=74548),
 FileInfo(path='dbfs:/FileStore/activations/2009-09.json', name='2009-09.json', size=78221),
 FileInfo(path='dbfs:/FileStore/activations/2009-10.json', name='2009-10.json', size=85751),
 FileInfo(path='dbfs:/FileStore/activations/2009-11.json', name='2009-11.json', size=87756),
 FileInfo(path='dbfs:/FileStore/activations/2009-12.json', name='2009-12.json', size=90772),
 FileInfo(path='dbfs:/FileStore/activations/2010-01.json', name='2010-01.json', size=181273),
 FileInfo(path='dbfs:/FileStore/activations/2010-02.json', name='2010-02.json', size=180555),
 FileInfo(path='dbfs:/FileStore/activations/2010-03.json', name='2010-03.json', size=189089),
 FileInfo(path='dbfs:/FileStore/activations/2010-04.json', name='2010-04.json', size=188924),
 FileInfo(path='dbfs:/FileStore/activations/2010-05.json', name='2010-05.json', size=213902),
 FileInfo(path='dbfs:/FileStore/activations/2010-06.json', name='2010-06.json', size=221724),
 FileInfo(path='dbfs:/FileStore/activations/2010-07.json', name='2010-07.json', size=216971),
 FileInfo(path='dbfs:/FileStore/activations/2010-08.json', name='2010-08.json', size=227790),
 FileInfo(path='dbfs:/FileStore/activations/2010-09.json', name='2010-09.json', size=220632),
 FileInfo(path='dbfs:/FileStore/activations/2010-10.json', name='2010-10.json', size=223382),
 FileInfo(path='dbfs:/FileStore/activations/2010-11.json', name='2010-11.json', size=197397),
 FileInfo(path='dbfs:/FileStore/activations/2010-12.json', name='2010-12.json', size=222284),
 FileInfo(path='dbfs:/FileStore/activations/2011-01.json', name='2011-01.json', size=416104),
 FileInfo(path='dbfs:/FileStore/activations/2011-02.json', name='2011-02.json', size=396798),
 FileInfo(path='dbfs:/FileStore/activations/2011-03.json', name='2011-03.json', size=445626),
 FileInfo(path='dbfs:/FileStore/activations/2011-04.json', name='2011-04.json', size=430525),
 FileInfo(path='dbfs:/FileStore/activations/2011-05.json', name='2011-05.json', size=439483),
 FileInfo(path='dbfs:/FileStore/activations/2011-06.json', name='2011-06.json', size=428603),
 FileInfo(path='dbfs:/FileStore/activations/2011-07.json', name='2011-07.json', size=439938),
 FileInfo(path='dbfs:/FileStore/activations/2011-08.json', name='2011-08.json', size=455153),
 FileInfo(path='dbfs:/FileStore/activations/2011-09.json', name='2011-09.json', size=437586),
 FileInfo(path='dbfs:/FileStore/activations/2011-10.json', name='2011-10.json', size=472058),
 FileInfo(path='dbfs:/FileStore/activations/2011-11.json', name='2011-11.json', size=449723),
 FileInfo(path='dbfs:/FileStore/activations/2011-12.json', name='2011-12.json', size=477450),
 FileInfo(path='dbfs:/FileStore/activations/2012-01.json', name='2012-01.json', size=923068),
 FileInfo(path='dbfs:/FileStore/activations/2012-02.json', name='2012-02.json', size=891243),
 FileInfo(path='dbfs:/FileStore/activations/2012-03.json', name='2012-03.json', size=952135),
 FileInfo(path='dbfs:/FileStore/activations/2012-04.json',

--> 9. Use the dbutils.fs.head command to view the format of one of the les. Although perhaps not formatted as well, it
should look something like this:

In [0]:
v_head = dbutils.fs.head("dbfs:/FileStore/activations/2008-11.json", 1000)
print(v_head)

[Truncated to first 1000 bytes]
{
 "activations": {
 "activation": [
 {
 "@timestamp": "1228068581",
 "@type": "phone",
 "account-number": "93893",
 "device-id": "b1d33f97-4f7f-4760-b6d5-2354b8ede7f9",
 "phone-number": "8313151801",
 "model": "Titanic 1100"
 },
 {
 "@timestamp": "1228064129",
 "@type": "phone",
 "account-number": "10524",
 "device-id": "e2e9a69e-9e9d-4cf0-a3be-b948c8f9bcd0",
 "phone-number": "3102505580",
 "model": "Titanic 1000"
 },
 {
 "@timestamp": "1228062743",
 "@type": "phone",
 "account-number": "116",
 "device-id": "f0f3bf10-4849-4409-a68b-d70be46729bc",
 "phone-number": "5101189988",
 "model": "Sorrento F00L"
 },
 {
 "@timestamp": "1228062084",
 "@type": "phone",
 "account-number": "428",
 "device-id": "fab0463c-aa81-4b0a-86dc-9b2dc336f9ea",
 "phone-number": "5106385454",
 "model": "Titan

## The task

Your code should process a set of activation JSON les and extract the account number and device model for each activation,
and save the list to a le formatted as account number:model

--> 10. Use wholeTextFiles to create an RDD from the activations dataset. The resulting RDD will consist of tuples, in
which the rst value is the name of the le, and the second value is the contents of the le (JSON) as a string.

In [0]:
AC_RDD = sc.wholeTextFiles("dbfs:/FileStore/activations/*.json")
AC_RDD.take(5)

Out[4]: [('dbfs:/FileStore/activations/2008-10.json',
 '{\n "activations": {\n "activation": [\n {\n "@timestamp": "1225462088",\n "@type": "phone",\n "account-number": "9763",\n "device-id": "debea35e-3ecd-4ee7-b0dd-ad428d953f32",\n "phone-number": "7600763387",\n "model": "MeeToo 1.0"\n },\n {\n "@timestamp": "1225461447",\n "@type": "phone",\n "account-number": "426",\n "device-id": "38a1566d-524e-4137-bad8-b597d09b54b0",\n "phone-number": "5102521038",\n "model": "Titanic 1000"\n },\n {\n "@timestamp": "1225446947",\n "@type": "phone",\n "account-number": "383",\n "device-id": "513024a3-a828-4674-9ff6-041e9a851f18",\n "phone-number": "9162206560",\n "model": "Sorrento F00L"\n },\n {\n "@timestamp": "1225445908",\n "@type": "phone",\n "account-number": "404",\n "device-id": "93d7d0eb-7551-4e11-ab59-91f34c04378f",\n "phone-number": "6506862748",\n "model": "MeeToo 1.0"\n },\n {\n "@timestamp": "1225444099",\n "@type": "phone",\n "account-number": "393",\n "device-id": "75faa3a2-31d6-48ec-a1a9-da4c882c7b75",\n "phone-number": "6504786402",\n "model": "iFruit 1"\n },\n {\n "@timestamp": "1225421991",\n "@type": "phone",\n "account-number": "53",\n "device-id": "a6482e48-d347-4ec5-b010-9d9a3b13d999",\n "phone-number": "6503155055",\n "model": "MeeToo 1.0"\n },\n {\n "@timestamp": "1225412236",\n "@type": "phone",\n "account-number": "96",\n "device-id": "351a0384-9af6-4621-ada8-3eba9fb2734c",\n "phone-number": "4152163825",\n "model": "iFruit 1"\n },\n {\n "@timestamp": "1225406348",\n "@type": "phone",\n "account-number": "283",\n "device-id": "04ad366c-c023-4d01-8493-8ec97d3ff5cc",\n "phone-number": "5109683453",\n "model": "Sorrento F00L"\n },\n {\n "@timestamp": "1225400017",\n "@type": "phone",\n "account-number": "464",\n "device-id": "e4ceefc4-8765-4beb-8b10-9b3d73eec535",\n "phone-number": "6504367851",\n "model": "MeeToo 1.0"\n },\n {\n "@timestamp": "1225386391",\n "@type": "phone",\n "account-number": "475",\n "device-id": "31fe0811-c4fb-40b8-bc59-8386fe2f5d8e",\n "phone-number": "6504313792",\n "model": "Titanic 1000"\n },\n {\n "@timestamp": "1225371984",\n "@type": "phone",\n "account-number": "479",\n "device-id": "38f76751-3236-4e99-80c9-c4b0a030108e",\n "phone-number": "6507811206",\n "model": "iFruit 1"\n },\n {\n "@timestamp": "1225355475",\n "@type": "phone",\n "account-number": "306",\n "device-id": "d743093b-8bcf-4ede-a475-4bc29936bc5e",\n "phone-number": "4158787317",\n "model": "Titanic 1000"\n },\n {\n "@timestamp": "1225349055",\n "@type": "phone",\n "account-number": "14045",\n "device-id": "1f568e8d-268a-4171-8a50-94c70fbfdf90",\n "phone-number": "9098824980",\n "model": "MeeToo 1.0"\n },\n {\n "@timestamp": "1225347793",\n "@type": "phone",\n "account-number": "6482",\n "device-id": "c14cbc3d-8b55-49bb-931d-7b2472aaa750",\n "phone-number": "4082887060",\n "model": "Sorrento F00L"\n },\n {\n "@timestamp": "1225345397",\n "@type": "phone",\n "account-number": "349",\n "device-id": "04c2fc8e-2f4f-4c7a-ad26-146c26deec87",\n "phone-number": "6502434241",\n "model": "MeeToo 1.0"\n },\n {\n "@timestamp": "1225333766",\n "@type": "phone",\n "account-number": "124",\n "device-id": "74d1341a-8da9-4de7-86f6-ec5285665ef2",\n "phone-number": "5103920538",\n "model": "MeeToo 1.0"\n },\n {\n "@timestamp": "1225333728",\n "@type": "phone",\n "account-number": "247",\n "device-id": "8ca7ba2f-5b8a-4574-abbb-de5a18221894",\n "phone-number": "6509604844",\n "model": "MeeToo 1.0"\n },\n {\n "@timestamp": "1225320127",\n "@type": "phone",\n "account-number": "88",\n "device-id": "9f7daca9-5faf-4cfd-b863-85d510464945",\n "phone-number": "6501458572",\n "model": "Sorrento F00L"\n },\n {\n "@timestamp": "1225319701",\n "@type": "phone",\n "account-number": "2939",\n "device-id": "18970270-86f0-4bac-ba03-24eb8518f082",\n "phone-number": "9161294398",\n "model": "Titanic 1000"\n },\n {\n "@timestamp": "1225317616",\n "@type": "phone",\n "account-number": "314",\n "device-id": "ae3cb61f-5318-4b74-bb05-9de91a66bb10",\n "phone-n

--> 11. Each JSON le can contain many activation records; map the contents of each le to a collection of JSON records.
Take each JSON string, parse it, and return a collection of JSON records; map each record to a separate RDD element.

In [0]:
import json 

AC_RDD2 = AC_RDD.map(lambda s: json.loads(s[1]))            #Map to make it a json view

AC_RDD2.take(1)

Out[37]: [{'activations': {'activation': [{'@timestamp': '1225462088',
 '@type': 'phone',
 'account-number': '9763',
 'device-id': 'debea35e-3ecd-4ee7-b0dd-ad428d953f32',
 'phone-number': '7600763387',
 'model': 'MeeToo 1.0'},
 {'@timestamp': '1225461447',
 '@type': 'phone',
 'account-number': '426',
 'device-id': '38a1566d-524e-4137-bad8-b597d09b54b0',
 'phone-number': '5102521038',
 'model': 'Titanic 1000'},
 {'@timestamp': '1225446947',
 '@type': 'phone',
 'account-number': '383',
 'device-id': '513024a3-a828-4674-9ff6-041e9a851f18',
 'phone-number': '9162206560',
 'model': 'Sorrento F00L'},
 {'@timestamp': '1225445908',
 '@type': 'phone',
 'account-number': '404',
 'device-id': '93d7d0eb-7551-4e11-ab59-91f34c04378f',
 'phone-number': '6506862748',
 'model': 'MeeToo 1.0'},
 {'@timestamp': '1225444099',
 '@type': 'phone',
 'account-number': '393',
 'device-id': '75faa3a2-31d6-48ec-a1a9-da4c882c7b75',
 'phone-number': '6504786402',
 'model': 'iFruit 1'},
 {'@timestamp': '1225421991',
 '@type': 'phone',
 'account-number': '53',
 'device-id': 'a6482e48-d347-4ec5-b010-9d9a3b13d999',
 'phone-number': '6503155055',
 'model': 'MeeToo 1.0'},
 {'@timestamp': '1225412236',
 '@type': 'phone',
 'account-number': '96',
 'device-id': '351a0384-9af6-4621-ada8-3eba9fb2734c',
 'phone-number': '4152163825',
 'model': 'iFruit 1'},
 {'@timestamp': '1225406348',
 '@type': 'phone',
 'account-number': '283',
 'device-id': '04ad366c-c023-4d01-8493-8ec97d3ff5cc',
 'phone-number': '5109683453',
 'model': 'Sorrento F00L'},
 {'@timestamp': '1225400017',
 '@type': 'phone',
 'account-number': '464',
 'device-id': 'e4ceefc4-8765-4beb-8b10-9b3d73eec535',
 'phone-number': '6504367851',
 'model': 'MeeToo 1.0'},
 {'@timestamp': '1225386391',
 '@type': 'phone',
 'account-number': '475',
 'device-id': '31fe0811-c4fb-40b8-bc59-8386fe2f5d8e',
 'phone-number': '6504313792',
 'model': 'Titanic 1000'},
 {'@timestamp': '1225371984',
 '@type': 'phone',
 'account-number': '479',
 'device-id': '38f76751-3236-4e99-80c9-c4b0a030108e',
 'phone-number': '6507811206',
 'model': 'iFruit 1'},
 {'@timestamp': '1225355475',
 '@type': 'phone',
 'account-number': '306',
 'device-id': 'd743093b-8bcf-4ede-a475-4bc29936bc5e',
 'phone-number': '4158787317',
 'model': 'Titanic 1000'},
 {'@timestamp': '1225349055',
 '@type': 'phone',
 'account-number': '14045',
 'device-id': '1f568e8d-268a-4171-8a50-94c70fbfdf90',
 'phone-number': '9098824980',
 'model': 'MeeToo 1.0'},
 {'@timestamp': '1225347793',
 '@type': 'phone',
 'account-number': '6482',
 'device-id': 'c14cbc3d-8b55-49bb-931d-7b2472aaa750',
 'phone-number': '4082887060',
 'model': 'Sorrento F00L'},
 {'@timestamp': '1225345397',
 '@type': 'phone',
 'account-number': '349',
 'device-id': '04c2fc8e-2f4f-4c7a-ad26-146c26deec87',
 'phone-number': '6502434241',
 'model': 'MeeToo 1.0'},
 {'@timestamp': '1225333766',
 '@type': 'phone',
 'account-number': '124',
 'device-id': '74d1341a-8da9-4de7-86f6-ec5285665ef2',
 'phone-number': '5103920538',
 'model': 'MeeToo 1.0'},
 {'@timestamp': '1225333728',
 '@type': 'phone',
 'account-number': '247',
 'device-id': '8ca7ba2f-5b8a-4574-abbb-de5a18221894',
 'phone-number': '6509604844',
 'model': 'MeeToo 1.0'},
 {'@timestamp': '1225320127',
 '@type': 'phone',
 'account-number': '88',
 'device-id': '9f7daca9-5faf-4cfd-b863-85d510464945',
 'phone-number': '6501458572',
 'model': 'Sorrento F00L'},
 {'@timestamp': '1225319701',
 '@type': 'phone',
 'account-number': '2939',
 'device-id': '18970270-86f0-4bac-ba03-24eb8518f082',
 'phone-number': '9161294398',
 'model': 'Titanic 1000'},
 {'@timestamp': '1225317616',
 '@type': 'phone',
 'account-number': '314',
 'device-id': 'ae3cb61f-5318-4b74-bb05-9de91a66bb10',
 'phone-number': '6509019772',
 'model': 'MeeToo 1.0'},
 {'@timestamp': '1225315853',
 '@type': 'phone',
 'account-number': '396',
 'device-id': '767d106f-4a25-4177-afe5-8f081fc68cd7',
 'phone-number': '6500312075',
 'model': 'MeeToo 1.0'},
 {'@timestamp': '1225278009',
 '@type': 'phone',
 'account

In [0]:
for record in AC_RDD2.take(1):
  internal_record = record["activations"]["activation"][:]
  for sub_rec in internal_record:
    print(sub_rec["account-number"] + ":" + sub_rec["model"])

9763:MeeToo 1.0
426:Titanic 1000
383:Sorrento F00L
404:MeeToo 1.0
393:iFruit 1
53:MeeToo 1.0
96:iFruit 1
283:Sorrento F00L
464:MeeToo 1.0
475:Titanic 1000
479:iFruit 1
306:Titanic 1000
14045:MeeToo 1.0
6482:Sorrento F00L
349:MeeToo 1.0
124:MeeToo 1.0
247:MeeToo 1.0
88:Sorrento F00L
2939:Titanic 1000
314:MeeToo 1.0
396:MeeToo 1.0
6881:MeeToo 1.0
234:iFruit 1
34209:Titanic 1000
369:MeeToo 1.0
255:iFruit 1
13:iFruit 1
142:Titanic 1000
11:iFruit 1
61:iFruit 1
1554:MeeToo 1.0
165:Titanic 1000
128224:MeeToo 1.0
394:MeeToo 1.0
433:MeeToo 1.0
448:iFruit 1
440:Sorrento F00L
240:MeeToo 1.0
460:MeeToo 1.0
275:Sorrento F00L
4638:Titanic 1000
171:Titanic 1000
213:Sorrento F00L
324:Sorrento F00L
98121:MeeToo 1.0
363:iFruit 1
52:Titanic 1000
286:iFruit 1
13702:Titanic 1000
345:MeeToo 1.0
36906:Titanic 1000
413:iFruit 1
459:Titanic 1000
144:Sorrento F00L
373:Titanic 1000
46849:Sorrento F00L
252:Titanic 1000
1:Titanic 1000
21566:Titanic 1000
154:Titanic 1000
46168:iFruit 1
3838:Titanic 1000
21821:MeeToo 1.0
204:iFruit 1
131:MeeToo 1.0
474:iFruit 1
146:Titanic 1000
2063:MeeToo 1.0
374:Sorrento F00L
89:Titanic 1000
194:Sorrento F00L
269:Titanic 1000
155:Sorrento F00L
405:MeeToo 1.0

--> 12. Map each activation record to a string in the format account-number:model

In [0]:
# This doesn't work because the object is "Dictionary of Dictionary of List of list of dictionary" basically. So, when multiple lists of lists are present, we use .flatMap transformation first. 
#ACC_MODEL_RDD = AC_RDD2.map(lambda record : record["activations"]["activation"]).map(lambda sub : sub[0]["account-number"])

#.flatMap() transformation - try it 
#ACC_MODEL_RDD = AC_RDD2.flatMap(lambda record : record["activations"]["activation"])

# In addition, we give a second map for extracting the required fields 
ACC_MODEL_RDD = AC_RDD2.flatMap(lambda record : record["activations"]["activation"]).map(lambda subrec : subrec["account-number"] + ":" + subrec["model"])
ACC_MODEL_RDD.take(15)

Out[102]: ['9763:MeeToo 1.0',
 '426:Titanic 1000',
 '383:Sorrento F00L',
 '404:MeeToo 1.0',
 '393:iFruit 1',
 '53:MeeToo 1.0',
 '96:iFruit 1',
 '283:Sorrento F00L',
 '464:MeeToo 1.0',
 '475:Titanic 1000',
 '479:iFruit 1',
 '306:Titanic 1000',
 '14045:MeeToo 1.0',
 '6482:Sorrento F00L',
 '349:MeeToo 1.0']

--> 13. Save the formatted strings to a text le in the DBFS directory /FileStore/account-models

In [0]:
try:                                                                                    # task 19 - Exception Handling to enable the notebook always run
  ACC_MODEL_RDD.saveAsTextFile("dbfs:/FileStore/account-models/")
except:
  print("File already exists in the path - dbfs:/FileStore/account-models --> So now we recursively delete this directory and create it again")
  dbutils.fs.rm("dbfs:/FileStore/account-models", True)
  ACC_MODEL_RDD.saveAsTextFile("dbfs:/FileStore/account-models")

In [0]:
ACC_MODEL_RDD.count()

Out[104]: 194764

In [0]:
print(dbutils.fs.head("dbfs:/FileStore/account-models/part-00000"))

[Truncated to first 65536 bytes]
9763:MeeToo 1.0
426:Titanic 1000
383:Sorrento F00L
404:MeeToo 1.0
393:iFruit 1
53:MeeToo 1.0
96:iFruit 1
283:Sorrento F00L
464:MeeToo 1.0
475:Titanic 1000
479:iFruit 1
306:Titanic 1000
14045:MeeToo 1.0
6482:Sorrento F00L
349:MeeToo 1.0
124:MeeToo 1.0
247:MeeToo 1.0
88:Sorrento F00L
2939:Titanic 1000
314:MeeToo 1.0
396:MeeToo 1.0
6881:MeeToo 1.0
234:iFruit 1
34209:Titanic 1000
369:MeeToo 1.0
255:iFruit 1
13:iFruit 1
142:Titanic 1000
11:iFruit 1
61:iFruit 1
1554:MeeToo 1.0
165:Titanic 1000
128224:MeeToo 1.0
394:MeeToo 1.0
433:MeeToo 1.0
448:iFruit 1
440:Sorrento F00L
240:MeeToo 1.0
460:MeeToo 1.0
275:Sorrento F00L
4638:Titanic 1000
171:Titanic 1000
213:Sorrento F00L
324:Sorrento F00L
98121:MeeToo 1.0
363:iFruit 1
52:Titanic 1000
286:iFruit 1
13702:Titanic 1000
345:MeeToo 1.0
36906:Titanic 1000
413:iFruit 1
459:Titanic 1000
144:Sorrento F00L
373:Titanic 1000
46849:Sorrento F00L
252:Titanic 1000
1:Titanic 1000
21566:Titanic 1000
154:Titanic 1000
46168:iFruit 1
3838:Titanic 1000
21821:MeeToo 1.0
204:iFruit 1
131:MeeToo 1.0
474:iFruit 1
146:Titanic 1000
2063:MeeToo 1.0
374:Sorrento F00L
89:Titanic 1000
194:Sorrento F00L
269:Titanic 1000
155:Sorrento F00L
405:MeeToo 1.0
93893:Titanic 1100
10524:Titanic 1000
116:Sorrento F00L
428:Titanic 1100
237:Titanic 1000
51:Titanic 1100
195:iFruit 1
38:Titanic 1100
147:Titanic 1100
120:Sorrento F00L
8386:Sorrento F00L
412:Sorrento F00L
458:MeeToo 1.0
14:Titanic 1100
265:Sorrento F00L
128:iFruit 1
257:iFruit 1
23566:Sorrento F00L
107295:Titanic 1000
27:iFruit 1
333:MeeToo 1.0
121392:MeeToo 1.0
335:Titanic 1000
923:MeeToo 1.0
41:iFruit 1
397:MeeToo 1.0
149:Sorrento F00L
476:Sorrento F00L
4:Titanic 1100
114:Sorrento F00L
189:iFruit 1
49:MeeToo 1.0
292:Titanic 1000
56:MeeToo 1.0
322:MeeToo 1.0
372:Titanic 1100
270:MeeToo 1.0
339:iFruit 1
2031:Titanic 1100
353:MeeToo 1.0
125:Sorrento F00L
99887:Titanic 1100
472:Sorrento F00L
8939:Sorrento F00L
370:iFruit 1
28434:Sorrento F00L
300:MeeToo 1.0
289:Sorrento F00L
151:iFruit 1
12066:Titanic 1000
22255:Sorrento F00L
364:iFruit 1
409:MeeToo 1.0
290:Titanic 1000
2603:Titanic 1000
96006:iFruit 1
414:Sorrento F00L
271:Titanic 1000
384:MeeToo 1.0
132:Titanic 1000
297:MeeToo 1.0
82:Titanic 1000
69821:Titanic 1000
175:iFruit 1
73:MeeToo 1.0
355:iFruit 1
26873:iFruit 1
299:Sorrento F00L
199:Sorrento F00L
449:MeeToo 1.0
356:iFruit 1
123:iFruit 1
467:Titanic 1000
407:Titanic 1000
463:iFruit 1
429:MeeToo 1.0
180:Sorrento F00L
12528:Titanic 1000
382:Titanic 1000
305:Sorrento F00L
293:Sorrento F00L
55861:iFruit 1
6386:Titanic 1000
79:Titanic 1000
6:Titanic 1000
241:Sorrento F00L
107404:iFruit 1
350:Titanic 1000
375:MeeToo 1.0
14501:Sorrento F00L
59:Titanic 1000
113:MeeToo 1.0
135:iFruit 1
28028:iFruit 1
432:Titanic 1000
403:Sorrento F00L
159:MeeToo 1.0
26001:Titanic 1000
68034:iFruit 1
262:Sorrento F00L
71:iFruit 1
25:MeeToo 1.0
29389:Titanic 1000
123408:Titanic 1000
215:MeeToo 1.0
284:Titanic 1000
223:iFruit 1
44:Titanic 1000
72:Sorrento F00L
134:Titanic 1000
337:MeeToo 1.0
376:Titanic 1000
185:MeeToo 1.0
207:MeeToo 1.0
307:Sorrento F00L
81:MeeToo 1.0
313:Sorrento F00L
386:Sorrento F00L
5:iFruit 1
58552:Sorrento F00L
112:Sorrento F00L
99:iFruit 1
32:Titanic 1000
139:iFruit 1
48:Sorrento F00L
2748:MeeToo 1.0
307:Sorrento F00L
1297:Titanic 1000
26:Titanic 1000
359:MeeToo 1.0
67:iFruit 1
366:Titanic 1000
60954:MeeToo 1.0
58:Sorrento F00L
22:Sorrento F00L
198:MeeToo 1.0
399:Titanic 1000
15:MeeToo 1.0
20132:iFruit 1
187:iFruit 1
77:iFruit 1
69:Sorrento F00L
427:Titanic 1000
59690:Titanic 1000
93:MeeToo 1.0
365:Titanic 1000
19:Titanic 1000
11036:Titanic 1000
228:iFruit 1
163:MeeToo 1.0
115:iFruit 1
232:Sorrento F00L
172:Sorrento F00L
227:Sorrento F00L
416:Titanic 1000
24:iFruit 1
120:Titanic 1000
98868:iFruit 1
203:iFruit 1
117312:Titanic 1000
73802:Sorrento F00L
74:MeeToo 1.0
230:MeeToo 1.0
209:MeeToo 1.0
2:iFruit 1
248:MeeToo 1.0
385:Sorrento F00L
55:Titanic 1000
35033:Titanic 1000
603:MeeToo 1.0
23:MeeToo 1.0
321:Sorrento F00L
45:iFruit 1
4